# Fetching air quality data from the FMI open data timeseries API

[API documentation](https://github.com/fmidev/smartmet-plugin-timeseries/blob/master/docs/Using-the-Timeseries-API.md),
[API examples](https://github.com/fmidev/smartmet-plugin-timeseries/blob/master/docs/Examples.md),
[JSON API example call](https://opendata.fmi.fi/timeseries?format=json&groupareas=0&producer=airquality_urban&area=Helsinki&param=time,fmisid,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg),
[CSV API call for the fmisid to name mapping](https://opendata.fmi.fi/timeseries?format=ascii&groupareas=0&separator=,&producer=airquality_urban&area=Finland&param=fmisid,name,latitude,longitude&starttime=2022-08-26T08:00:00%2B00:00&endtime=2022-08-26T08:00:00%2B00:00&tz=UTC).

In [1]:
import requests
import pendulum
import pandas as pd
import numpy as np

In [2]:
start_time = pendulum.yesterday('UTC')
start_time = pendulum.now('UTC').subtract(days=5)
end_time = pendulum.tomorrow('UTC')

aq_fields = {
    'fmisid': np.int32,
    'time': np.datetime64,
    'AQINDEX_PT1H_avg': np.float64,
    'PM10_PT1H_avg': np.float64,
    'PM25_PT1H_avg': np.float64,
    'O3_PT1H_avg': np.float64,
    'CO_PT1H_avg': np.float64,
    'SO2_PT1H_avg': np.float64,
    'NO2_PT1H_avg': np.float64,
    'TRSC_PT1H_avg': np.float64,
}

url = 'https://opendata.fmi.fi/timeseries'

params = {
    'format': 'json',
    'precision': 'double',
    'groupareas': '0',
    'producer': 'airquality_urban',
    'area': 'Uusimaa',
    'param': ','.join(aq_fields.keys()),
    'starttime': start_time.isoformat(timespec="seconds"),
    'endtime': end_time.isoformat(timespec="seconds"),
    'tz': 'UTC',
}

data = requests.get(url, params=params).json()

In [3]:
df = pd.DataFrame(data).astype(aq_fields)
# df = df.set_index(['fmisid', 'time'])
df[0:10]

,fmisid,time,AQINDEX_PT1H_avg,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg
0,100662,2022-08-23 10:00:00,2.0,8.7,4.9,72.6,NaN,0.7,6.8,NaN
1,100662,2022-08-23 11:00:00,2.0,2.4,3.9,75.8,NaN,0.6,6.3,NaN
2,100662,2022-08-23 12:00:00,2.0,-3.2,2.9,78.1,NaN,0.7,5.2,NaN
3,100662,2022-08-23 13:00:00,2.0,7.0,3.8,72.6,NaN,0.8,10.4,NaN
4,100662,2022-08-23 14:00:00,2.0,17.4,4.7,74.1,NaN,0.6,10.7,NaN
5,100662,2022-08-23 15:00:00,2.0,11.5,6.2,70.9,NaN,0.7,10.0,NaN
6,100662,2022-08-23 16:00:00,2.0,14.9,6.1,63.0,NaN,0.9,12.8,NaN
7,100662,2022-08-23 17:00:00,2.0,15.6,5.3,67.8,NaN,0.7,7.2,NaN
8,100662,2022-08-23 18:00:00,2.0,0.5,5.4,67.9,NaN,0.7,7.5,NaN
9,100662,2022-08-23 19:00:00,2.0,6.5,6.2,65.9,NaN,0.8,13.5,NaN


In [4]:
df.to_parquet('data/airquality.parquet', compression='zstd')

# DuckDB

In [5]:
import duckdb
con = duckdb.connect(database=':memory:')

In [6]:
con.execute('CREATE OR REPLACE TABLE airquality_urban AS SELECT * FROM df')

In [7]:
df2 = con.execute('SELECT * FROM airquality_urban').fetchdf()
df2.sample(10)

,fmisid,time,AQINDEX_PT1H_avg,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg
1535,104083,2022-08-27 12:00:00,2.0,18.9,10.2,NaN,NaN,NaN,5.0,NaN
1573,107147,2022-08-24 02:00:00,1.0,14.0,7.6,NaN,NaN,NaN,3.4,NaN
348,100723,2022-08-27 22:00:00,1.0,8.3,6.5,1.6,NaN,0.5,6.6,NaN
1847,107400,2022-08-25 12:00:00,1.0,10.4,NaN,NaN,NaN,NaN,7.3,NaN
1762,107399,2022-08-26 23:00:00,1.0,10.6,6.6,NaN,NaN,NaN,12.8,NaN
1226,104048,2022-08-24 15:00:00,2.0,23.5,12.2,NaN,NaN,NaN,12.8,NaN
205,100691,2022-08-26 23:00:00,1.0,10.3,7.2,NaN,NaN,NaN,14.4,NaN
449,100742,2022-08-27 03:00:00,1.0,13.8,7.9,NaN,NaN,NaN,23.6,NaN
250,100723,2022-08-23 20:00:00,1.0,9.7,4.3,15.2,NaN,0.5,6.1,NaN
880,103139,2022-08-25 02:00:00,1.0,NaN,NaN,30.6,NaN,-0.3,1.2,NaN
